# Tracer une carte en Python avec bokeh

[bokeh](https://bokeh.pydata.org/en/latest/) permet de tracer une carte sur laquelle on peut zoomer, dézoomer et qui ne dépend pas d'un service extérieur comme [folium](http://folium.readthedocs.io/en/latest/).

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## données

In [2]:
from papierstat.datasets import load_enedis_dataset
df = load_enedis_dataset()
df.head(n=2).T

,0,1
Année,2016,2016
Nom commune,Ponteilla,Varreddes
Code commune,66145,77483
Nom EPCI,CU Perpignan Méditerranée (Pmcu),CA Pays de Meaux
Code EPCI,200027183,247700628
Type EPCI,CU,CA
Nom département,Pyrénées-Orientales,Seine-et-Marne
Code département,66,77
Nom région,Occitanie,Île-de-France
Code région,76,11


## cartes avec bokeh

[bokeh](https://bokeh.pydata.org/en/latest/) n'inclut aucune données géographiques, il faut les récupérer. Les pays sont disponibles à cette adresse [github.com/johan](https://github.com/johan/world.geo.json).

In [3]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource
from bokeh.models import Range1d
from papierstat.datasets import get_geojson_countries

with open(get_geojson_countries(), "r") as f:
    countries = GeoJSONDataSource(geojson=f.read())

tools = "pan,wheel_zoom,reset,previewsave"
p = figure(width=1000, height=600, tools=tools, title='World Countries', 
           x_axis_label='Longitude', y_axis_label='Latitude')
p.background_fill_color = "aqua"
p.x_range = Range1d(start=-180, end=180)
p.y_range = Range1d(start=-90, end=90)
p.patches("xs", "ys", color="white", line_color="black", source=countries)
show(p)

Et plus spécifique autour de la France avec quelques animations.

In [5]:
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
from bokeh.models.glyphs import Patches

tools = "pan,wheel_zoom,reset,previewsave"
p = figure(width=500, height=400, tools=tools, title='World Countries', 
           x_axis_label='Longitude', y_axis_label='Latitude')
p.background_fill_color = "aqua"
p.x_range = Range1d(start=-5, end=10)
p.y_range = Range1d(start=42, end=52)

pat1 = Patches(xs="xs", ys="ys", fill_color="white", line_color="black")
p.add_glyph(countries, pat1)

communes = ColumnDataSource(data=dict(xc=df.long, yc=df.lat,
                                      commune=df['Nom commune'],
                                      tension=df['Domaine de tension']))
p.scatter(x="xc", y="yc", fill_color="blue", line_color="blue", source=communes)

hover = HoverTool(tooltips=[("->", "@commune"), ('T', "@tension")],
                  mode="mouse", point_policy="follow_mouse")
p.add_tools(hover)

show(p)